In [1]:
import pandas as pd
import numpy as np
import joblib
import plotnine as pn
from xgboost import XGBClassifier, XGBRegressor
from sklearn.model_selection import GridSearchCV

In [2]:
pn.options.dpi = 300

In [3]:
cdnow_raw_df = pd.read_csv('../data/cdnow.csv')